In [1]:
# Wine Quality Sample

def train(in_alpha, in_l1_ratio):
    import os
    import warnings
    import sys

    import pandas as pd
    import numpy as np
    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
    from sklearn.model_selection import train_test_split
    from sklearn.linear_model import ElasticNet

    import mlflow
    import mlflow.sklearn
    
    import logging
    logging.basicConfig(level=logging.WARN)
    logger = logging.getLogger(__name__)

    def eval_metrics(actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2


    warnings.filterwarnings("ignore")
    np.random.seed(40)

    # Read the wine-quality csv file from the URL
    csv_url =\
        'http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
    try:
        data = pd.read_csv(csv_url, sep=';')
    except Exception as e:
        logger.exception(
            "Unable to download training & test CSV, check your internet connection. Error: %s", e)

    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]

    # Set default values if no alpha is provided
    if float(in_alpha) is None:
        alpha = 0.5
    else:
        alpha = float(in_alpha)

    # Set default values if no l1_ratio is provided
    if float(in_l1_ratio) is None:
        l1_ratio = 0.5
    else:
        l1_ratio = float(in_l1_ratio)

    # Useful for multiple runs (only doing one run in this sample notebook)    
    with mlflow.start_run():
        # Execute ElasticNet
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        # Evaluate Metrics
        predicted_qualities = lr.predict(test_x)
        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        # Print out metrics
        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        # Log parameter, metrics, and model to MLflow
        mlflow.log_param("alpha", alpha)
        mlflow.log_param("l1_ratio", l1_ratio)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)

        mlflow.sklearn.log_model(lr, "model")

In [2]:
train(0.5, 0.5)

2020/02/08 08:57:18 WARNING mlflow.tracking.context.git_context: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh()

All git commands will error until this is rectified.

This initial warning can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|none|n|0: for no warning or exception
    - warn|w|warning|1: for a printed warning
    - error|e|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet



Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.7931640229276851
  MAE: 0.6271946374319587
  R2: 0.10862644997792614


In [3]:
train(1,1)

Elasticnet model (alpha=1.000000, l1_ratio=1.000000):
  RMSE: 0.8328280500742669
  MAE: 0.672504249193216
  R2: 0.017246911564584355


In [4]:
train(1, 0.5)

Elasticnet model (alpha=1.000000, l1_ratio=0.500000):
  RMSE: 0.8328241218819784
  MAE: 0.6681571546815985
  R2: 0.017256182227355366


In [5]:
train(0.5, 1)

Elasticnet model (alpha=0.500000, l1_ratio=1.000000):
  RMSE: 0.832819092896359
  MAE: 0.6681279771237894
  R2: 0.017268050734704277


In [6]:
train(0.25, 1)

Elasticnet model (alpha=0.250000, l1_ratio=1.000000):
  RMSE: 0.7851146844348285
  MAE: 0.619283241012321
  R2: 0.12662666187495986


In [7]:
train(0.5, .25)

Elasticnet model (alpha=0.500000, l1_ratio=0.250000):
  RMSE: 0.7596554775612442
  MAE: 0.5913132541174235
  R2: 0.18235068599935988


In [8]:
train(0.75, 1)

Elasticnet model (alpha=0.750000, l1_ratio=1.000000):
  RMSE: 0.8329952816627894
  MAE: 0.6706323229041399
  R2: 0.016852198969261267


In [9]:
train(1, 0.25)

Elasticnet model (alpha=1.000000, l1_ratio=0.250000):
  RMSE: 0.8030971071368318
  MAE: 0.637004229900228
  R2: 0.08616065531997052


In [10]:
train(1, 0.75)

Elasticnet model (alpha=1.000000, l1_ratio=0.750000):
  RMSE: 0.8329943914543187
  MAE: 0.6706401496400851
  R2: 0.016854300316176185


In [11]:
train(0.75, 0.75)

Elasticnet model (alpha=0.750000, l1_ratio=0.750000):
  RMSE: 0.8331799787336064
  MAE: 0.669234506901795
  R2: 0.016416170929074214


In [12]:
train(0.25, 0.25)

Elasticnet model (alpha=0.250000, l1_ratio=0.250000):
  RMSE: 0.7380489682487518
  MAE: 0.5690312554727687
  R2: 0.2282012262646781


In [13]:
train(0.0, 0.25)

Elasticnet model (alpha=0.000000, l1_ratio=0.250000):
  RMSE: 0.6603811224461619
  MAE: 0.5073644199888925
  R2: 0.3820931219110161


In [14]:
train(0.25, 0)

Elasticnet model (alpha=0.250000, l1_ratio=0.000000):
  RMSE: 0.7202072749674797
  MAE: 0.5538448306277172
  R2: 0.26506532688734685


In [15]:
train(0.1, 0.1)

Elasticnet model (alpha=0.100000, l1_ratio=0.100000):
  RMSE: 0.7128829045893679
  MAE: 0.5462202174984665
  R2: 0.2799376066653344


In [16]:
train(0.001, 0.001)

Elasticnet model (alpha=0.001000, l1_ratio=0.001000):
  RMSE: 0.6624915696582605
  MAE: 0.5090087948753361
  R2: 0.3781373952488176


In [17]:
train(0, 0)

Elasticnet model (alpha=0.000000, l1_ratio=0.000000):
  RMSE: 0.6603811224461619
  MAE: 0.5073644199888925
  R2: 0.3820931219110161
